<a href="https://colab.research.google.com/github/oforiPrince/Challenge-1/blob/main/Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dash

In [10]:
# Required Libraries
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Load the predictions data
file_path = "/content/xgboost_predictions_2024.csv"
predictions_df = pd.read_csv(file_path)

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Pitch Mix Predictions by Player for 2024 MLB Season"),
    dcc.Dropdown(
        id='player-dropdown',
        options=[{'label': player, 'value': player} for player in predictions_df['PLAYER_NAME'].unique()],
        value=predictions_df['PLAYER_NAME'].iloc[0],  # Default value
        multi=False
    ),
    dcc.Graph(id='pitch-type-pie-chart'),
    dcc.Graph(id='pitch-type-bar-chart'),
    html.Button("Save Dashboard", id="save-button", n_clicks=0)
])

# Callback to update the pie chart based on selected player
@app.callback(
    Output('pitch-type-pie-chart', 'figure'),
    Input('player-dropdown', 'value')
)
def update_pie_chart(selected_player):
    # Filter the data for the selected player
    player_data = predictions_df[predictions_df['PLAYER_NAME'] == selected_player].iloc[0]
    # Create a pie chart
    fig = px.pie(
        names=['Fastball', 'Breaking Ball', 'Off-Speed'],
        values=[player_data['PITCH_TYPE_FB'], player_data['PITCH_TYPE_BB'], player_data['PITCH_TYPE_OS']],
        title=f'Expected Pitch Mix for {selected_player}'
    )
    return fig

# Callback to update the bar chart based on selected player
@app.callback(
    Output('pitch-type-bar-chart', 'figure'),
    Input('player-dropdown', 'value')
)
def update_bar_chart(selected_player):
    # Filter the data for the selected player
    player_data = predictions_df[predictions_df['PLAYER_NAME'] == selected_player].iloc[0]
    # Prepare data for the bar chart
    pitch_types = ['Fastball', 'Breaking Ball', 'Off-Speed']
    pitch_values = [player_data['PITCH_TYPE_FB'], player_data['PITCH_TYPE_BB'], player_data['PITCH_TYPE_OS']]

    # Create a bar chart
    fig = px.bar(
        x=pitch_types,
        y=pitch_values,
        title=f'Pitch Type Probabilities for {selected_player}',
        labels={'x': 'Pitch Type', 'y': 'Probability'},
        color=pitch_types
    )
    return fig

# Save dashboard functionality
@app.callback(
    Output("save-button", "children"),
    Input("save-button", "n_clicks")
)
def save_dashboard(n):
    if n > 0:
        # Save the dashboard to an HTML file
        with open("pitch_predictions_dashboard.html", "w") as f:
            f.write(app.index_string)
        return "Dashboard saved!"
    return "Save Dashboard"

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>